In [1]:
import time
import sep
import glob
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
import astroalign as aa
from astropy.io import fits
from astropy.stats import SigmaClip
from astropy.coordinates import SkyCoord
from photutils import CircularAperture
from photutils.centroids import centroid_sources
from photutils.aperture import aperture_photometry
from photutils.background import Background2D, MedianBackground
from astropy.wcs.utils import skycoord_to_pixel, pixel_to_skycoord
from astropy.coordinates import ICRS, FK5
from regions import CirclePixelRegion, PixCoord
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Bunch of empty arrays for various things. They all live up here for conveniance. Most are used, some aren't and are leftover from older versions/things I tried.
hdus = []
aa_images = []

idex = []
cents = []
info = []
comb = []
masked_ims = []
bsims = []
residuals = []
cpu = []
ram = []

In [3]:
c = 0 #counter 
mask_count = 0 #another counter
start = time.time() #time tracker for how long the "pipeline" runs.

outstr = r"C:\Users\Sam Whitebook\Documents\FitsOutput\Files\res{}.fits.fz" #write out string for residuals, edit for your filepath. 
#IN MY EXTREMELY RIGOUROUS TESTING, I'VE FOUND IT'S BEST TO USE SETS WITH MORE THAN 100 IMAGES.
files = sorted(glob.glob(r"C:\Users\Sam Whitebook\Documents\Visual Studio 2010\Projects\Lubin Lab\Data\KOI-5156_IN\*.fz")) #science directory, sorted.

In [4]:
def mask(img, srcs, c): #masking function. Takes science image, list of sources (positions), and counter from above. 
    dat = img
    empty_mask = np.zeros([dat.shape[0]+100, dat.shape[1]+100]) # zeros array with the dimensions of the science image +100 pixels. The additional buffer allows masking of edge sources, this is cropped out later
    blend_mask = np.ones([dat.shape[0]+100, dat.shape[1]+100]) #ones array with dimensions of science image + 100pixels
    
    for i, s in enumerate(srcs):
        x_cen = s['x']
        y_cen = s['y']
        radius = ((s['xmax'] - s['xmin'])/2) + 3 #adds a bit to the radius to avoid leaving a halo/fringe on the outer edge
        #print(x_cen, y_cen, radius)
        center_2 = PixCoord(x_cen+50, y_cen+50)

        circle = CirclePixelRegion(center_2, radius) 
        mask = circle.to_mask() #this and the previous line make a "circle" array with ones
        app2 = CircularAperture([x_cen , y_cen] , radius) 
        app_phot2 = aperture_photometry(dat, app2) #aperture sum
        blend_val = app_phot2['aperture_sum'][0]/(np.pi*radius**2) #averaging the sum value over the area of the source
        empty_mask[mask.bbox.slices] += mask*blend_val #applies each mask times the averaged value to the empty frame
        blend_mask[mask.bbox.slices] -= mask #applies the mask to the ones array, essentially leaving negative space where each source is (not actually negative values but negative space in the desgin sense)
        
    dlt = np.arange(0, 50, 1) #cropping for the zeros array
    del1 = np.delete(empty_mask, [i for i in dlt], axis=0)
    del2 = np.delete(del1, [i +dat.shape[0] for i in dlt], axis=0)
    del3 = np.delete(del2, [i for i in dlt], axis=1)
    del4 = np.delete(del3, [i + dat.shape[1] for i in dlt], axis=1)

    del1a = np.delete(blend_mask, [i for i in dlt], axis=0) #cropping for the ones array
    del2a = np.delete(del1a, [i +dat.shape[0] for i in dlt], axis=0)
    del3a = np.delete(del2a, [i for i in dlt], axis=1)
    del4a = np.delete(del3a, [i + dat.shape[1] for i in dlt], axis=1)

    masked_im = np.multiply(dat, del4a) #applies negative space (blend_mask) to science image, zeroing out the original sources
    blended_im  = masked_im + del4 #appalies psf averaged sources back to the original frame.
    return blended_im

In [6]:
for fit in files: #actual start of code, opens files
    hdu = fits.open(fit)
    hdus.append(hdu)

sigma_clip = SigmaClip(sigma=3.0)
bkg_estimator = MedianBackground()
aa_images.append((hdus[0])[1].data) #adds first image in set separately since we are aligning everything else to that image; i.e. it doesn't makes sense to align an image to itself. 
for im in hdus[1:]:#aligns all images in hdus except the first, to the first
    aligned = aa.register(im[1].data, (hdus[0])[1].data)[0] #align each image (except the first) to the first image in the set. aa.register returns aligned and footprint [0] index returns aligned
    bkg = Background2D(aligned, (50, 50), filter_size=(3, 3), sigma_clip=sigma_clip, bkg_estimator=bkg_estimator)
    bkg_median = bkg.background_median
    subtracted = aligned - bkg_median
    aa_images.append(subtracted)


#bkg0 = Background2D(hdus[0][1].data, (50, 50), filter_size=(3, 3), sigma_clip=sigma_clip, bkg_estimator=bkg_estimator)
bkg0 = sep.Background((hdus[0])[1].data) #background sub
extracted0 = sep.extract((hdus[0])[1].data -bkg0.back(), bkg0.globalrms *3, minarea = 5, segmentation_map = False) #source extractor run since the LCO BANZAI SEP run has weird settings and didn't pick up enough sources

tracker = 0 #counter for debugging. 
for img in aa_images:
    masked_ims.append(mask(img, extracted0, c)) #runs each aligned image through masking
    #masked_ims.append(img)
    #print(tracker)
    tracker +=1
c = -1 #allows you to run masking differently for the templeate if you want
template = np.median(aa_images, axis = 0) #makes template
blend_template = mask(template, extracted0, c) #masks template

In [7]:
#just writes out the masked and unmasked templates so they are available to look at
temp_out = fits.PrimaryHDU(template) 
temp_out.writeto(r"C:\Users\Sam Whitebook\Documents\FitsOutput\Templates\7imgtemplate2.fits")
btemp_out = fits.PrimaryHDU(blend_template)
btemp_out.writeto(r"C:\Users\Sam Whitebook\Documents\FitsOutput\Templates\7imgtemplateblend2.fits")

for msk in masked_ims:
    residuals.append(np.subtract(msk, blend_template))
    #residuals.append(ois.optimal_system(image=msk, refimage=template, method='Bramich')[0])

#another SEP run to pick out sources from the residuals. In theory, these will be a certain set value from the median template and worth doing photometry on.
counter = 0
for pics in residuals:
    bkg = sep.Background(pics)
    extracted = sep.extract(pics -bkg.back(), bkg.globalrms *3, minarea = 20, segmentation_map = False)
    header = fits.Header([fits.Card("History", "Extracted by sep")])
    hdul = fits.HDUList([fits.PrimaryHDU(pics), fits.BinTableHDU(data = extracted, header = header, name ="SEP", ver  = None)]) 
    hdul.writeto(outstr.format(counter))
    counter += 1

end = time.time()
elapsed = end - start
msg = "Done in: {} seconds"
print(msg.format(elapsed))
#print(mask_count)

Done in: 2842.7009222507477 seconds
